In [2]:
import pandas as pd

dataset = '/Users/xavierphilipponneau/Downloads/btcusd_1-min_data.csv'
df = pd.read_csv(dataset)
df.head()

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [4]:
from datetime import datetime
import backtrader as bt
import pandas as pd

dataset = '/Users/xavierphilipponneau/Downloads/btcusd_1-min_data.csv'

def get_data():
    # Charger les données CSV
    df = pd.read_csv(dataset)

    # Adapter les colonnes au format attendu par Backtrader
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='s')
    df = df.rename(columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    })
    df = df[['Date', 'open', 'high', 'low', 'close', 'volume']]
    df = df.set_index('Date')

    # Feed pour Backtrader
    data = bt.feeds.PandasData(dataname=df)
    return data


class EquityLogger(bt.Analyzer):
    def __init__(self):
        self.equity = []

    def next(self):
        self.equity.append({'datetime': self.strategy.datas[0].datetime.datetime(0),
                            'value': self.strategy.broker.getvalue()})

    def get_analysis(self):
        return self.equity


class GridLeverageStrategy(bt.Strategy):
    params = (('gap', 500), ('leverage', 20),)

    def __init__(self):
        self.open_long_order = None
        self.close_long_order = None
        self.size = 0

    def start(self):
        self.cash = self.broker.get_cash()
        self.size = (self.cash * self.params.leverage) / self.data.close[0]

    def next(self):
        price = self.data.close[0]

        # Placer les ordres si inexistant
        if not self.open_long_order and not self.position:
            self.open_long_order = self.buy(price=price - self.params.gap, size=self.size, exectype=bt.Order.Limit)
            self.close_long_order = self.sell(price=price + self.params.gap, size=self.size, exectype=bt.Order.Limit)

    def notify_order(self, order):
        if order.status == order.Completed:
            executed_price = order.executed.price

            # Si open long exécuté
            if order == self.open_long_order:
                # Place un nouveau close long à partir du nouveau prix
                self.close_long_order = self.sell(price=executed_price + self.params.gap, size=self.size, exectype=bt.Order.Limit)
                self.open_long_order = self.buy(price=executed_price - self.params.gap, size=self.size, exectype=bt.Order.Limit)

            # Si close long exécuté
            elif order == self.close_long_order:
                # Remet un nouveau open long + close long à partir du prix de clôture
                self.close_long_order = self.sell(price=executed_price + self.params.gap, size=self.size, exectype=bt.Order.Limit)
                # Supprime l'ancien open long (Backtrader ne permet pas d'annuler un order exécuté, donc rien à faire ici)
                self.open_long_order = self.buy(price=executed_price - self.params.gap, size=self.size, exectype=bt.Order.Limit)

cerebro = bt.Cerebro()
# Remplacez par vos propres données
#data = bt.feeds.YahooFinanceData(dataname='BTC', fromdate=datetime(2020,1,1), todate=datetime(2024,1,1))
data = get_data()
cerebro.adddata(data)
cerebro.addstrategy(GridLeverageStrategy, gap=500, leverage=20)
# cerebro.addanalyzer(EquityLogger, _name='equitylog')
cerebro.run()
cerebro.plot()


/Users/xavierphilipponneau/Projets/hyperliquid/.venv/lib/python3.12/site-packages/backtrader/cerebro.py:670: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the
/Users/xavierphilipponneau/Projets/hyperliquid/.venv/lib/python3.12/site-packages/backtrader/cerebro.py:712: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the


ImportError: Matplotlib seems to be missing. Needed for plotting support